In [2]:
import pandas as pd
import os

companies_data = pd.read_csv("../data/companies.csv")
articles_data = pd.read_csv("../data/articles_about_breaches.csv")

companies_data['name_lower'] = companies_data['name'].str.lower()
articles_data['CompanyMentioned_lower'] = articles_data['CompanyMentioned'].str.lower()

# Perform the join on the lowercase columns
result_df = pd.merge(companies_data, articles_data, left_on='name_lower', right_on='CompanyMentioned_lower', how='inner')

# Drop the helper lowercase columns if desired
result_df = result_df.drop(columns=['name_lower', 'CompanyMentioned_lower'])

result_df.to_csv("../data/joined_articles_companies.csv", index=False)